### ライブラリインポート

In [1]:
import numpy as np
from tqdm import tqdm
from numba import jit, f8, c16, i8
import yaml
import os
import sys
os.makedirs('output', exist_ok=True)
os.makedirs('output/wavefunction1', exist_ok=True)
os.makedirs('output/wavefunction2', exist_ok=True)

### 関数

In [2]:
def vor_add(x,y,si,position_x,position_y):
    return np.exp(1j*si*np.arctan2(x-position_x,y-position_y))

def vortex_create(vor_num,R_0,x,y):
    #print(type(vor_num),type(R_0),type(x),type(y))
    count_vor_num:int=0
    pos_x=np.zeros(vortex_num,dtype='float32')
    pos_y=np.zeros(vortex_num,dtype='float32')
    while count_vor_num<vor_num:
        x_tmp=(np.random.rand()-0.5)*R_0*2
        y_tmp=(np.random.rand()-0.5)*R_0*2
        if x_tmp**2+y_tmp**2<R_0**2:
            pos_x[count_vor_num]=x_tmp
            pos_y[count_vor_num]=y_tmp
            count_vor_num=count_vor_num+1
    sign=np.ones(vor_num,dtype=int)
    sign[0:vor_num//2]=-1
    phi=1
    for i in range(len(pos_x)):
        phi=phi*vor_add(x-np.max(x)/2,y-np.max(y)/2,sign[i],pos_x[i],pos_y[i])
        phi=phi*vor_add(x+np.max(x)/2,y-np.max(y)/2,-sign[i],-pos_x[i],pos_y[i])
        phi=phi*vor_add(x-np.max(x)/2,y+np.max(y)/2,-sign[i],pos_x[i],-pos_y[i])
        phi=phi*vor_add(x+np.max(x)/2,y+np.max(y)/2,sign[i],-pos_x[i],-pos_y[i])
    return phi

def aliasing(targ,K):
    filter_aliasing=K<np.max(K)/8
    return targ*filter_aliasing

def RungeKutta(nonlinear_func,target,target_sub,alpha,V,G,delta_t,K,vol):
    k_1=nonlinear_func(target,target_sub,V,G,K,vol)
    k_1=aliasing(k_1,K)
    k_2=nonlinear_func(np.exp(-alpha*delta_t/2)*(target+delta_t*k_1/2),np.exp(-alpha*delta_t/2)*(target_sub+delta_t*k_1/2),V,G,K,vol)
    k_2=aliasing(k_2,K)
    k_3=nonlinear_func(np.exp(-alpha*delta_t/2)*target+delta_t*k_2/2,np.exp(-alpha*delta_t/2)*target_sub+delta_t*k_2/2,V,G,K,vol)
    k_3=aliasing(k_3,K)
    k_4=nonlinear_func(np.exp(-alpha*delta_t)*target+np.exp(-alpha*delta_t/2)*delta_t*k_3,np.exp(-alpha*delta_t)*target_sub+np.exp(-alpha*delta_t/2)*delta_t*k_3,V,G,K,vol)
    k_4=aliasing(k_4,K)
    new_target=np.exp(-alpha*delta_t)*target+delta_t*(np.exp(-alpha*delta_t)*k_1+2*np.exp(-alpha*delta_t/2)*k_2+2*np.exp(-alpha*delta_t/2)*k_3+k_4)/6
    return new_target
    
def nonlinear_image(Psi_k,Psi_k_sub,V,G,K,vol):
    Psi_1=np.fft.ifft2(Psi_k).astype(np.complex64)
    Psi_2=np.fft.ifft2(Psi_k_sub).astype(np.complex64)
    return np.fft.fft2(-V*Psi_1-vol*np.abs(Psi_1)**2*Psi_1-G*vol*np.abs(Psi_2)**2*Psi_1)

def nonlinear_real(Psi_k,Psi_k_sub,V,G,K,vol):
    Psi_1=np.fft.ifft2(Psi_k).astype(np.complex64)
    Psi_2=np.fft.ifft2(Psi_k_sub).astype(np.complex64)
    return np.fft.fft2(-1j*V*Psi_1-1j*vol*np.abs(Psi_1)**2*Psi_1-1j*G*vol*np.abs(Psi_2)**2*Psi_1)

### 空間生成

In [3]:
with open('config.yaml') as file:
    object_yaml = yaml.safe_load(file)
    space_div_num=object_yaml['space_div_num']
    a_osc_num=object_yaml['a_osc_num']
    delta_t=object_yaml['delta_t']
    t_N=object_yaml['t_N']
    g_12=object_yaml['g_12']
    vortex_num=object_yaml['vortex_num']
    R_0=object_yaml['R_0']
    V_0=object_yaml['V_0']
    output_wavefunction=object_yaml['output_wavefunction']
    Nonlinearity=object_yaml['Nonlinearity']
delta_x=1/a_osc_num

space=np.linspace(-delta_x*(space_div_num-2)/2,delta_x*(space_div_num)/2,space_div_num).astype(np.float32)
x,y=np.meshgrid(space,space)
space_sq=x**2+y**2
k=2*np.pi*np.fft.fftfreq(space_div_num,d=delta_x)
k=k.astype(np.float32)
k_x,k_y=np.meshgrid(k,k)
k_sq=k_x**2+k_y**2
print("Spatial resolution : "+str(delta_x))
print("Wavenumber resolution : "+str(k[1]))
print("Time resolution : "+str(delta_t))

H=(x-np.min(x)/2)**2+(y-np.min(y)/2)**2>(R_0*0.9)**2
H=H*((x+np.min(x)/2)**2+(y-np.min(y)/2)**2>(R_0*0.9)**2)
H=H*((x-np.min(x)/2)**2+(y+np.min(y)/2)**2>(R_0*0.9)**2)
H=H*((x+np.min(x)/2)**2+(y+np.min(y)/2)**2>(R_0*0.9)**2)
V=(np.tanh(0.6*(((x-np.min(x)/2)**2+(y-np.min(y)/2)**2)**0.5-1.12*R_0))+1)
V=V*(np.tanh(0.6*(((x+np.min(x)/2)**2+(y-np.min(y)/2)**2)**0.5-1.12*R_0))+1)
V=V*(np.tanh(0.6*(((x-np.min(x)/2)**2+(y+np.min(y)/2)**2)**0.5-1.12*R_0))+1)
V=V*(np.tanh(0.6*(((x+np.min(x)/2)**2+(y+np.min(y)/2)**2)**0.5-1.12*R_0))+1)
V=V*V_0
V_k=np.fft.fft2(V)
V=np.fft.ifft2(aliasing(V_k,k_sq))
V=V.astype(np.complex64)

Spatial resolution : 0.14285714285714285
Wavenumber resolution : 0.04295146
Time resolution : 0.01


### 波動関数生成

In [4]:
psi_1=vortex_create(vortex_num,R_0,x,y)
psi_1=psi_1/(np.sum(np.abs(psi_1)**2)*delta_x**2)**0.5

psi_2=vortex_create(vortex_num,R_0,x,y)
psi_2=psi_2/(np.sum(np.abs(psi_2)**2)*delta_x**2)**0.5

psi_k_1=aliasing(np.fft.fft2(psi_1),k_sq)
psi_k_2=aliasing(np.fft.fft2(psi_2),k_sq)
print("initial wavefunction is created")

initial wavefunction is created


### 虚時間発展

In [5]:
linear_term_imag=k_sq/2
for i in tqdm(range(10)):
    psi_k_1_tmp=RungeKutta(nonlinear_image,psi_k_1,psi_k_2,linear_term_imag,V,g_12,delta_t,k_sq,Nonlinearity)
    psi_k_2=RungeKutta(nonlinear_image,psi_k_2,psi_k_1,linear_term_imag,V,g_12,delta_t,k_sq,Nonlinearity)
    psi_k_1=psi_k_1_tmp
    psi_k_1=aliasing(psi_k_1,k_sq)
    psi_k_2=aliasing(psi_k_2,k_sq)
psi_1=np.fft.ifft2(psi_k_1)
psi_1=psi_1/(np.sum(np.abs(psi_1)**2)*delta_x**2)**0.5
psi_k_1=aliasing(np.fft.fft2(psi_1),k_sq)
psi_2=np.fft.ifft2(psi_k_2)
psi_2=psi_2/(np.sum(np.abs(psi_2)**2)*delta_x**2)**0.5
psi_k_2=aliasing(np.fft.fft2(psi_2),k_sq)
np.save('output/wavefunction1/initial',psi_1)
np.save('output/wavefunction2/initial',psi_2)

100%|██████████| 10/10 [00:14<00:00,  1.41s/it]


### 実時間発展

In [6]:
psi_1=np.fft.ifft2(psi_k_1)
psi_2=np.fft.ifft2(psi_k_2)
linear_term_real=1j*k_sq/2
for i in tqdm(range(t_N)):
    psi_k_1_tmp=RungeKutta(nonlinear_real,psi_k_1,psi_k_2,linear_term_real,V,g_12,delta_t,k_sq,Nonlinearity)
    psi_k_2=RungeKutta(nonlinear_real,psi_k_2,psi_k_1,linear_term_real,V,g_12,delta_t,k_sq,Nonlinearity)
    psi_k_1=psi_k_1_tmp
    psi_k_1=aliasing(psi_k_1,k_sq)
    psi_k_2=aliasing(psi_k_2,k_sq)
    if (i+1)%output_wavefunction==0:
        psi_1=np.fft.ifft2(psi_k_1)
        psi_2=np.fft.ifft2(psi_k_2)
        np.save('output/wavefunction1/'+str(i+1),psi_1)
        np.save('output/wavefunction2/'+str(i+1),psi_2)
        del psi_1,psi_2

100%|██████████| 1000/1000 [31:57<00:00,  1.92s/it]
